In [ ]:
import os
import re
import cv2
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import face_recognition

from cv2 import dnn_superres

In [ ]:
# MODEL_PATH = "/home/serverai/Project/face-recognition/face_attendance_system/prod/models/result.json"
# MODEL_PATH = "/home/serverai/Project/face-recognition/face_attendance_system/prod/models/encode_dev.json"
MODEL_PATH = "/home/serverai/Project/face-recognition/face_attendance_system/prod/models/result-rev2.json"
# MODEL_PATH = "result.json"

In [ ]:
with open(MODEL_PATH) as file:
    face_encode = json.load(file)
list_face_encode = [data[0] for data in face_encode.values()]

In [ ]:
imgs_new = 145
res = 141

In [ ]:
len(face_encode)

In [ ]:
success_test = 'images/success/'
failed_test = 'images/failed/'

## Necessary Function

In [ ]:
def check_accuracy(path):
#     list_all = []
    for image in os.listdir(path):
        if image != ".ipynb_checkpoints":
            print(image)
            img_path = f"{path}/{image}"
            
            img = cv2.imread(img_path)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            face_locations = face_recognition.face_locations(img)
            encode_image = face_recognition.face_encodings(img, face_locations)[0]
            face_distances = face_recognition.face_distance(list_face_encode, encode_image)
            valid_distances = sorted((dist, idx) for idx, dist in enumerate(face_distances) if dist < float(0.35))

            if valid_distances:
                closest_faces_indices = [idx for _, idx in valid_distances[:5]]
                distance = [dist for dist,_ in valid_distances[:5]]

                candidates = []
                for idx, dist in zip(closest_faces_indices, distance):
                    candidate_key = next(key for key, value in face_encode.items() if np.array_equal(value[0], list_face_encode[idx]))
                    score = round((1 - dist) * 100, 2)
                    candidates.append((candidate_key, score))
                print(candidates)
            else:
                print("No valid distances")
            print("=====")
#     df = pd.DataFrame(list_all, columns=['name', 'score'])
#     return candidates

In [ ]:
path = "images/error2"
check_accuracy(path)

In [ ]:
def plot_images(images, figsize=(10, 10)):
    scores = []
    ncol = 5
    nrow = len(images) // 5
    fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=figsize)
    axes = axes.flatten()
    for ax, img in zip(axes, images):
        candidates = get_face_loc(img)
        scores.append(candidates)
#         print(f'\n {candidates}')
        ax.imshow(img, cmap='gray')
        ax.axis('off')
    
#     print("\n")
    df = create_df(scores)
    print(df)
    average = get_avg_value(df)
    
    plt.tight_layout()
    plt.show()
    
    return average

In [ ]:
def get_face_loc(img_arr):
    
    candidates = []
    
    try:
        face_locations = face_recognition.face_locations(img_arr)
        if not face_locations:
            candidates = []
        else:
            encode_image = face_recognition.face_encodings(img_arr, face_locations, num_jitters=2, model='large')[0]
            face_distances = face_recognition.face_distance(list_face_encode, encode_image)
            valid_distances = sorted((dist, idx) for idx, dist in enumerate(face_distances) if dist < float(0.35))

            if valid_distances:
                closest_faces_indices = [idx for _, idx in valid_distances[:5]]
                distance = [dist for dist,_ in valid_distances[:5]]

                for idx, dist in zip(closest_faces_indices, distance):
                    candidate_key = next(key for key, value in face_encode.items() if np.array_equal(value[0], list_face_encode[idx]))
                    score = round((1 - dist) * 100, 2)
                    candidates.append((candidate_key, score))
    except:
        print('This image does not contain face(s)')
                
    return candidates

In [ ]:
def create_df(a_list):
    df_flat = [item for sublist in a_list for item in sublist]
    df = pd.DataFrame(df_flat, columns=['name', 'score'])
    
    return df

In [ ]:
def get_avg_value(df):
    df['name'] = [name.split('-')[0] for name in df.name]
    df = df.groupby('name')['score'].mean().reset_index()
    df.columns = ['name', 'average_score']
    
    return df

## Original vs Flip Images

In [ ]:
def check_rgb_score(img_path):
    candidates = []
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(img_rgb)
    print(face_locations)
    if face_locations:
        encode_image = face_recognition.face_encodings(img_rgb, face_locations, num_jitters=2, model='large')[0]
        face_distances = face_recognition.face_distance(list_face_encode, encode_image)
        valid_distances = sorted((dist, idx) for idx, dist in enumerate(face_distances) if dist < float(0.36))

        if valid_distances:
            closest_faces_indices = [idx for _, idx in valid_distances[:5]]
            distance = [dist for dist,_ in valid_distances[:5]]

            for idx, dist in zip(closest_faces_indices, distance):
                candidate_key = next(key for key, value in face_encode.items() if np.array_equal(value[0], list_face_encode[idx]))
                score = round((1 - dist) * 100, 2)
                candidates.append((candidate_key, score))
                
        print(f"valid indices \n {candidates}")
    else:
        print(f"no valid indices")

    return candidates

In [ ]:
def check_bgr_score(img_path):
    candidates = []
    img = cv2.imread(img_path)
    face_locations = face_recognition.face_locations(img)
    print(face_locations)
    if face_locations:
        encode_image = face_recognition.face_encodings(img, face_locations, num_jitters=2, model='large')[0]
        face_distances = face_recognition.face_distance(list_face_encode, encode_image)
        valid_distances = sorted((dist, idx) for idx, dist in enumerate(face_distances) if dist < float(0.36))

        if valid_distances:
            closest_faces_indices = [idx for _, idx in valid_distances[:5]]
            distance = [dist for dist,_ in valid_distances[:5]]

            for idx, dist in zip(closest_faces_indices, distance):
                candidate_key = next(key for key, value in face_encode.items() if np.array_equal(value[0], list_face_encode[idx]))
                score = round((1 - dist) * 100, 2)
                candidates.append((candidate_key, score))
    
        print(f" valid indices \n {candidates}")
    else:
        print(f"no valid indices")

    return candidates

In [ ]:
def load_scores(folder_path):
    bgr_list = []
    rgb_list = []
    
    for img_path in os.listdir(folder_path):
        images = os.path.join(folder_path, img_path)
        start_time = time.time()
        print(f"\n image {images} on BGR")
        bgr_scores = check_bgr_score(images)
        print(f"Process time: {round((time.time() - start_time), 2)} \n ====")
        bgr_list.append(bgr_scores)
        start_time = time.time()
        print(f"\n image {images} on RGB")
        rgb_scores = check_rgb_score(images)
        print(f"Process time: {round((time.time() - start_time), 2)} \n ====")
        rgb_list.append(rgb_scores)
    
    return bgr_list, rgb_list

In [ ]:
bgr_score, rgb_score = load_scores(failed_test)

In [ ]:
df_rgb = create_df(rgb_score)
df_rgb

In [ ]:
get_avg_value(df_rgb)

In [ ]:
get_avg_value(df_rgb)['average_score'].mean().round(2)

In [ ]:
df_bgr = create_df(bgr_score)
df_bgr

In [ ]:
get_avg_value(df_bgr)

In [ ]:
get_avg_value(df_bgr)['average_score'].mean().round(2)

In [ ]:
def check_flip_rgb_score(img_path):
    candidates = []
    img = cv2.imread(img_path)
    flip = cv2.flip(img, 1)
    img_rgb = cv2.cvtColor(flip, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(img_rgb)
    if face_locations:
        encode_image = face_recognition.face_encodings(img_rgb, face_locations)[0]
        face_distances = face_recognition.face_distance(list_face_encode, encode_image)
        valid_distances = sorted((dist, idx) for idx, dist in enumerate(face_distances) if dist < float(0.35))

        if valid_distances:
            closest_faces_indices = [idx for _, idx in valid_distances[:5]]
            distance = [dist for dist,_ in valid_distances[:5]]

            for idx, dist in zip(closest_faces_indices, distance):
                candidate_key = next(key for key, value in face_encode.items() if np.array_equal(value[0], list_face_encode[idx]))
                score = round((1 - dist) * 100, 2)
                candidates.append((candidate_key, score))
        print(f"valid indices \n {candidates}")
    else:
        print(f"no valid indices")
#     print("====")
    return candidates

In [ ]:
def check_flip_bgr_score(img_path):
    candidates = []
    img = cv2.imread(img_path)
    flip = cv2.flip(img, 1)
    face_locations = face_recognition.face_locations(flip)
    if face_locations:
        encode_image = face_recognition.face_encodings(flip, face_locations)[0]
        face_distances = face_recognition.face_distance(list_face_encode, encode_image)
        valid_distances = sorted((dist, idx) for idx, dist in enumerate(face_distances) if dist < float(0.35))

        if valid_distances:
            closest_faces_indices = [idx for _, idx in valid_distances[:5]]
            distance = [dist for dist,_ in valid_distances[:5]]

            for idx, dist in zip(closest_faces_indices, distance):
                candidate_key = next(key for key, value in face_encode.items() if np.array_equal(value[0], list_face_encode[idx]))
                score = round((1 - dist) * 100, 2)
                candidates.append((candidate_key, score))
        print(f"valid indices \n {candidates}")
    else:
        print(f"no valid indices")
#     print("====")

    return candidates

In [ ]:
def load_flip_scores(folder_path):
    bgr_list = []
    rgb_list = []
    
    for img_path in os.listdir(folder_path):
        
        images = os.path.join(folder_path, img_path)
        start_time = time.time()
        print(f"\n image {images} on BGR")
        bgr_scores = check_flip_bgr_score(images)
        print(f"Process time: {round((time.time() - start_time), 2)} \n ====")
        bgr_list.append(bgr_scores)
        start_time = time.time()
        print(f"\n image {images} on RGB")
        rgb_scores = check_flip_rgb_score(images)
        print(f"Process time: {round((time.time() - start_time), 2)} \n ====")
        rgb_list.append(rgb_scores)
    
    return bgr_list, rgb_list

In [ ]:
flip_bgr_score, flip_rgb_score = load_flip_scores(failed_test)

In [ ]:
df_flip_rgb = create_df(flip_rgb_score)
df_flip_rgb

In [ ]:
get_avg_value(df_flip_rgb)

In [ ]:
get_avg_value(df_flip_rgb)['average_score'].mean().round(2)

In [ ]:
df_flip_bgr = create_df(flip_bgr_score)
df_flip_bgr

In [ ]:
get_avg_value(df_flip_bgr)

In [ ]:
get_avg_value(df_flip_bgr)['average_score'].mean().round(2)

In [ ]:
test = 'images/31#1-revasya/failed/'

In [ ]:
score = []

for img_path in os.listdir(test):
    print(img_path)
    path = os.path.join(test, img_path)
#     print(path)
    rgb_score = check_rgb_score(path)
    score.append(rgb_score)

In [ ]:
ori_imgs = [cv2.imread(os.path.join(failed_test, img)) for img in os.listdir(failed_test)]
# img_rgb = [cv2.cvtColor(cv2.imread(os.path.join(failed_test, img)), cv2.COLOR_BGR2RGB) for img in os.listdir(failed_test)]
ori_avg = plot_images(ori_imgs)
ori_avg

In [ ]:
print(f"Mean Average Score for Original Images: \n{ori_avg['average_score'].mean().round(2)}")

In [ ]:
# ori_imgs = [cv2.imread(os.path.join(success_test, img)) for img in os.listdir(success_test)]
img_rgb = [cv2.cvtColor(cv2.imread(os.path.join(failed_test, img)), cv2.COLOR_BGR2RGB) for img in os.listdir(failed_test)]
ori_avg = plot_images(img_rgb)
ori_avg

In [ ]:
flip_imgs = []
for img_path in os.listdir(failed_test):
#     images = cv2.imread(os.path.join(failed_test, img_path))
    images = cv2.cvtColor(cv2.imread(os.path.join(failed_test, img_path)), cv2.COLOR_BGR2RGB)
    flip = cv2.flip(images, 1)
    cv2.cvtColor(flip, cv2.COLOR_BGR2RGB)
    flip_imgs.append(flip)
flip_avg = plot_images(flip_imgs)
flip_avg

In [ ]:
print(f"Mean Average Score for Flip Original Images: \n{flip_avg['average_score'].mean().round(2)}")

# Normalize image

In [ ]:
def only_show_images(images, figsize=(10, 10)):
    scores = []
    ncol = 5
    nrow = len(images) // 5
    fig, axes = plt.subplots(nrows=nrow, ncols=ncol, figsize=figsize)
    axes = axes.flatten()
    for ax, img in zip(axes, images):
        ax.imshow(img, cmap='gray')
        ax.axis('off')
    

    plt.tight_layout()
    plt.show()

In [ ]:
test = 'images/test/'
img_rgb = [cv2.cvtColor(cv2.imread(os.path.join(test, img)), cv2.COLOR_BGR2RGB) for img in os.listdir(test)]
for i in img_rgb:
    print(get_face_loc(i))
only_show_images(img_rgb)

In [ ]:
fliped = []
img_rgb = [cv2.cvtColor(cv2.imread(os.path.join(test, img)), cv2.COLOR_BGR2RGB) for img in os.listdir(test)]
for i in img_rgb:
    i = cv2.flip(i, 1)
    fliped.append(i)
    print(get_face_loc(i))
only_show_images(fliped)

In [ ]:
upsampling = []
for i in sr_imgs:
    rows, cols, _channels = map(int, i.shape)
    ups = cv2.pyrUp(i, dstsize=(2 * cols, 2 * rows))
    cv2.cvtColor(ups, cv2.COLOR_BGR2RGB)
    upsampling.append(ups)
    print(get_face_loc(ups))
only_show_images(upsampling)

In [ ]:

norm_imgs = []
for i in sr_imgs:
# #     images = cv2.imread(os.path.join(failed_test, img_path))
#     images = cv2.cvtColor(cv2.imread(os.path.join(test, img_path)), cv2.COLOR_BGR2RGB)
    normi = cv2.normalize(i, None, alpha=0, beta=355, norm_type=cv2.NORM_MINMAX)
    cv2.cvtColor(normi, cv2.COLOR_BGR2RGB)
    norm_imgs.append(normi)
only_show_images(norm_imgs)

In [ ]:
for i in norm_imgs:
    print(get_face_loc(i))

## Sharpness Enhancement

In [ ]:
def enhance_sharpness(image, kernel):
    sharp_image = cv2.filter2D(image, -1, kernel)
    return sharp_image

In [ ]:
kernels = {
    'mild': np.array([
        [0, -0.5, 0],
        [-0.5, 3, -0.5],
        [0, -0.5, 0]
    ]),
    'normal': np.array([
        [0, -1, 0],
        [-1, 5, -1],
        [0, -1, 0]
    ])
}

In [ ]:
sharp_normal_img = []
for i in norm_imgs:
    start_time = time.time()
    sharpen = enhance_sharpness(i, kernels['mild'])
    print(f"Process image in time: {round((time.time() - start_time), 2)} \n ====")
    sharp_normal_img.append(sharpen)
sn_avg = plot_images(sharp_normal_img)
sn_avg

In [ ]:
print(f"Mean Average Score for Normal Sharp Flip Images: \n{sn_avg['average_score'].mean().round(2)}")

In [ ]:
sharp_mild_img = []
for i in flip_imgs:
    start_time = time.time()
    sharpen = enhance_sharpness(i, kernels['mild'])
    print(f"Process image in time: {round((time.time() - start_time), 2)} \n ====")
    sharp_mild_img.append(sharpen)
sm_avg = plot_images(sharp_mild_img)
sm_avg

In [ ]:
print(f"Mean Average Score for Mild Sharp Flip Images: \n{sm_avg['average_score'].mean().round(2)}")

## Auto Brightness

In [ ]:
limit = {
    'low_contrast': 0.5,
    'medium_contrast': 0.75,
    'high_contrast': 1.0,
    'xtra_contrast': 2.0
}

In [ ]:
def auto_brightness(img, limit):
    
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=limit, tileGridSize=(4, 4))
    cl = clahe.apply(l)

    limg = cv2.merge((cl, a, b))
    result = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    return result

 ### from Original Flip

In [ ]:
flip_brightness = []

for i in norm_imgs:
    start_time = time.time()
    lab = auto_brightness(i, limit['medium_contrast'])
    print(f"Process image in time: {round((time.time() - start_time), 2)} \n ====")
    flip_brightness.append(lab)
auto_flip_avg = plot_images(flip_brightness)
auto_flip_avg

In [ ]:
print(f"Mean Average Score for Brighter Original Images: \n{auto_flip_avg['average_score'].mean().round(2)}")

 ### from Sharp Normal Flip

In [ ]:
sn_brightness = []

for i in sharp_normal_img:
    start_time = time.time()
    lab = auto_brightness(i, limit['medium_contrast'])
    print(f"Process image in time: {round((time.time() - start_time), 2)} \n ====")
    sn_brightness.append(lab)
auto_sn_avg = plot_images(sn_brightness)
auto_sn_avg

In [ ]:
print(f"Mean Average Score for Brighter Sharp Normal Images: \n{auto_sn_avg['average_score'].mean().round(2)}")

 ### from Sharp Mild Flip

In [ ]:
sm_brightness = []

for i in sharp_mild_img:
    start_time = time.time()
    lab = auto_brightness(i, limit['medium_contrast'])
    print(f"Process image {i} in time: {round((time.time() - start_time), 2)} \n ====")
    sm_brightness.append(lab)
auto_sm_avg = plot_images(sm_brightness)
auto_sm_avg

In [ ]:
print(f"Mean Average Score for Brighter Sharp Normal Images: \n{auto_sm_avg['average_score'].mean().round(2)}")

In [ ]:
image = cv2.imread('path_to_your_image.jpg')

def upscale(img, scale=2)
scale_factor = 2

# Get the original dimensions
height, width = image.shape[:2]

# Compute new dimensions
new_width = int(width * scale_factor)
new_height = int(height * scale_factor)

# Resize the image
resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)


# Another Test

In [ ]:
img_path = "images/1722210090_.jpg"

In [ ]:
img = cv2.imread(img_path)
img = cv2.flip(img, 1)

In [ ]:
img_lab = auto_brightness(img, limit['medium_contrast'])
img_rgb = cv2.cvtColor(img_lab, cv2.COLOR_BGR2RGB)

In [ ]:
face_locations = face_recognition.face_locations(img_lab)

In [ ]:
encode_image = face_recognition.face_encodings(img_lab, face_locations)[0]
encode_image

In [ ]:
face_distances = face_recognition.face_distance(list_face_encode, encode_image)
face_distances

In [ ]:
valid_distances = sorted((dist, idx) for idx, dist in enumerate(face_distances) if dist < float(0.36))
valid_distances

In [ ]:
for idx, dist in enumerate(face_distances):
    if dist < float(0.45):
        print(sorted((dist, idx)))

In [ ]:
check_rgb_score('images/1722317218_JORGI VEGASERRA PARFIN.jpg')

In [ ]:
check_bgr_score('images/1722317218_JORGI VEGASERRA PARFIN.jpg')

In [ ]:
check_flip_rgb_score('images/1722317218_JORGI VEGASERRA PARFIN.jpg')

In [ ]:
check_flip_bgr_score('images/1722317218_JORGI VEGASERRA PARFIN.jpg')

In [ ]:
i = auto_brightness(img_rgb, limit['high_contrast'])
fig, axes = plt.subplots(1, 1, figsize=(5, 5))

axes.imshow(i, cmap='gray')
axes.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def get_spesific_encode(name, dic):
    result = []

    def search_dict(d, name=name):
        for key, value in d.items():
            all_names = re.findall(r'\w+', key)[1]
            if name in all_names:
                print
                result.append({key: value})
            if isinstance(value, dict):
                search_dict(value)

    search_dict(dic)
    
    return result

In [ ]:
revasya = get_spesific_encode('REVASYA', face_encode)

In [ ]:
revasya

# Advanced Preprocessing

In [ ]:
img_src = cv2.imread('images/good-img/NUGROHO.jpg')
img_src_rgb = cv2.cvtColor(img_src, cv2.COLOR_BGR2RGB)

In [ ]:
rows, cols, _channels = map(int, img_src_rgb.shape)
img_src_rgb.shape

In [ ]:
plt.imshow(img_src_rgb)
plt.show()

In [ ]:
get_face_loc(img_src_rgb)

In [ ]:
ups = cv2.cvtColor(cv2.pyrUp(img_src, dstsize=(2 * cols, 2 * rows)), cv2.COLOR_BGR2RGB)

ups.shape

In [ ]:
plt.imshow(ups)
plt.show()

In [ ]:
get_face_loc(ups)

In [ ]:
rows, cols, _channels = map(int, ups.shape)
ups.shape

In [ ]:
ups2 = cv2.cvtColor(cv2.pyrUp(ups, dstsize=(2 * cols, 2 * rows)), cv2.COLOR_BGR2RGB)
ups2 = cv2.cvtColor(ups2, cv2.COLOR_BGR2RGB)
ups2.shape

In [ ]:
plt.imshow(ups2)
plt.show()

In [ ]:
get_face_loc(ups2)

In [ ]:
gaussian = cv2.GaussianBlur(norm_img, (5, 5), 0)
plt.imshow(gaussian)
plt.show()

In [ ]:
norm_img = cv2.normalize(lab, None, alpha=0, beta=355, norm_type=cv2.NORM_MINMAX)
print(norm_img.shape)

plt.imshow(norm_img)
plt.show()

In [ ]:
get_face_loc(norm_img)

In [ ]:
lab = auto_brightness(result, limit['medium_contrast'])
print(lab.shape)

plt.imshow(lab)
plt.show()

In [ ]:
get_face_loc(lab)

In [ ]:
sharpen = enhance_sharpness(lab, kernels['mild'])
cv2.imwrite("images/sharpness.jpg", sharpen)
plt.imshow(sharpen)
plt.show()

In [ ]:
get_face_loc(sharpen)

In [ ]:
lab

In [ ]:
image = 'images/test/1721434068_SHAFIRA NINDY HAPSARI.jpg'

In [ ]:
img = cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB)
print(img.shape)
plt.imshow(img)
plt.show()

In [ ]:
superres = dnn_superres.DnnSuperResImpl_create()

In [ ]:
path = ['weights/FSRCNN-small_x2.pb', 'weights/FSRCNN-small_x3.pb', 'weights/FSRCNN-small_x4.pb', 
        'weights/FSRCNN_x2.pb', 'weights/FSRCNN_x3.pb', 'weights/FSRCNN_x4.pb']

In [ ]:
superres.readModel(path[3])

superres.setModel("fsrcnn", 2)

In [ ]:
# Upscale the image
start_time = time.time()
result = superres.upsample(img)

print(f"Process image in time: {round((time.time() - start_time), 2)} \n ====")


In [ ]:
print(result.shape)
plt.imshow(result)
plt.show()

In [ ]:
def superres(image, scale):
    path = 'weights/FSRCNN_x2.pb'
    
    superres = dnn_superres.DnnSuperResImpl_create()
    superres.readModel(path)
    superres.setModel("fsrcnn", scale)
    
    start_time = time.time()
    result = superres.upsample(image)

    print(f"Process image in time: {round((time.time() - start_time), 2)} \n ====")
    
    return result

In [ ]:
test = 'images/test/'

ori_imgs = []
for img_path in os.listdir(test):
#     images = cv2.imread(os.path.join(failed_test, img_path))
    images = cv2.cvtColor(cv2.imread(os.path.join(test, img_path)), cv2.COLOR_BGR2RGB)
    ori_imgs.append(images)
only_show_images(ori_imgs)

In [ ]:
for i in ori_imgs:
    print(get_face_loc(i))

In [ ]:
test = 'images/test/'

sr_imgs = []
for i in ori_imgs:
    sr_img = superres(i, 2)
    sr_imgs.append(sr_img)
only_show_images(sr_imgs)

In [ ]:
for i in sr_imgs:
    print(get_face_loc(i))

# Testing

In [ ]:
import io
import imageio

def save_to(img_arr, src_path):
    filename = f"{src_path.split('.')[0]}_changed.JPG"
    image_file = io.BytesIO()
    imageio.imwrite(image_file, img_arr, format='jpg')
    image_file.seek(0)

    with open(filename, 'wb') as f:
        f.write(image_file.getbuffer())
        

In [ ]:
def preprocessing(img_path):
    start_time = time.time()

    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    path = 'weights/FSRCNN_x3.pb'
    
    superres = dnn_superres.DnnSuperResImpl_create()
    superres.readModel(path)
    superres.setModel("fsrcnn", 3)
    
    img_rgb = superres.upsample(img_rgb)
    
    img_rgb = cv2.normalize(img_rgb, None, alpha=0, beta=320, norm_type=cv2.NORM_MINMAX)
    cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    
#     img_rgb = enhance_sharpness(img_rgb, kernels['mild'])
#     img_rgb = cv2.GaussianBlur(img_rgb, (3, 3), 0)
    
    print(f"Process {img_path} in time: {round((time.time() - start_time), 2)} \n ====")
    save_to(img_rgb, img_path)
    
    return img_rgb

In [ ]:
dir_test = "images/error2"

final = []
for imgs in os.listdir(dir_test):
    img_path = os.path.join(dir_test, imgs)
    preprocessed = preprocessing(img_path)
    final.append(preprocessed)

In [ ]:
for i in final:
    print(get_face_loc(i))

In [ ]:
def downscale_image(image, scale):
    width, height = image.size
    return image.resize((width // scale, height // scale), Image.BICUBIC)

def prepare_data(data_dir, output_dir, scale=4):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img = Image.open(os.path.join(data_dir, filename))
            lr_img = downscale_image(img, scale)
            lr_img.save(os.path.join(output_dir, filename))

In [ ]:
data_dirs = 'images/failed'
lowr_dirs = 'images/downsample'

prepare_data(data_dirs, lowr_dirs)

In [ ]:
ori_imgs = [cv2.imread(os.path.join(lowr_dirs, img)) for img in os.listdir(lowr_dirs)]
# img_rgb = [cv2.cvtColor(cv2.imread(os.path.join(latest, img)), cv2.COLOR_BGR2RGB) for img in os.listdir(latest)]
ori_avg = plot_images(ori_imgs)
ori_avg

In [ ]:
# Generator
def build_generator():
    def residual_block(input):
        x = Conv2D(64, (3, 3), padding='same')(input)
        x = BatchNormalization(momentum=0.8)(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(64, (3, 3), padding='same')(x)
        x = BatchNormalization(momentum=0.8)(x)
        return tf.keras.layers.add([input, x])
    
    input = Input(shape=(None, None, 3))
    x = Conv2D(64, (9, 9), padding='same')(input)
    x = LeakyReLU(alpha=0.2)(x)
    
    for _ in range(16):
        x = residual_block(x)
    
    x = UpSampling2D(size=2)(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(3, (9, 9), padding='same')(x)
    output = tf.keras.layers.Activation('tanh')(x)
    
    return Model(input, output)

generator = build_generator()
generator.compile(optimizer=Adam(0.0002, 0.5), loss='mse')
generator.summary()
